In [18]:
import pandas as pd

test_results = pd.read_csv("test_results.txt")
test_results.drop(test_results.columns[0], axis=1, inplace=True)
user_table = pd.read_csv("user_table.txt")


df = test_results.merge(user_table, on ='user_id')





In [19]:
df.drop(columns = ['lat', 'long', 'user_id', 'country'], inplace=True)
df['operative_system']=df.groupby('device').operative_system.transform(lambda x: x.fillna(x.mode()[0]))
df = df.fillna(df.mode().iloc[0])
print(df['timestamp'][0])

df['timestamp'] =df['timestamp'].str.replace('60', '00')
df['timestamp'] =pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S', errors = 'coerce')
print(df['timestamp'][0].hour)
print(type(df['timestamp'][0]))


df['hour'] = df['timestamp'].dt.hour
df['month'] = df['timestamp'].dt.month
df['weekday'] = df['timestamp'].dt.weekday
df['year'] = df['timestamp'].dt.year


2015-05-08 03:38:34
3
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [20]:
#check for seasonality
print(df['month'].value_counts())
#check how historical the data is
print(df['year'].value_counts())
print('hour value counts',df['hour'].value_counts())
print('weekday value counts',df['weekday'].value_counts())

df.drop(columns = ['month', 'year', 'timestamp'], inplace=True)

#check trial users were actually displayed 59

print(len(df[df['test']==1]))
print(len(df[df['price']==59.0]))

#remove inconsistent rows
df = df[(df['test']==1) & (df['price']==59.0) | (df['test']==0) & (df['price']==39.0)]

print(len(df))
df.drop(columns = ['price'], inplace=True)
#remove hours and weekdays that don't make sense
df = df.dropna()

5.0    99072
4.0    88840
3.0    87440
Name: month, dtype: int64
2015.0    275352
Name: year, dtype: int64
hour value counts 12.0    33364
11.0    29060
10.0    24336
9.0     19270
13.0    14882
16.0    14594
8.0     14415
14.0    14345
18.0    14308
17.0    14206
15.0    14168
7.0      9584
2.0      5069
5.0      5014
22.0     4979
3.0      4979
1.0      4969
0.0      4950
21.0     4890
19.0     4844
20.0     4843
6.0      4814
23.0     4782
4.0      4687
Name: hour, dtype: int64
weekday value counts 4.0    79888
5.0    64514
6.0    48706
3.0    32119
1.0    17066
0.0    16684
2.0    16375
Name: weekday, dtype: int64
99188
98244
272621


In [ ]:
#Fisher's exact test'








In [21]:
from sklearn import preprocessing
df_enc = df.drop(columns=['test'])
to_encode = ['source', 'device', 'operative_system', 'city']
le = preprocessing.LabelEncoder()

for i in to_encode:
    df_enc[i] = le.fit_transform(df_enc[i])


In [ ]:
from kmodes.kmodes import KModes

cost=[]
for k in list(range(2,11)):
        print('Iteration number:', k)
        km = KModes(n_clusters=k, n_init = 1, verbose=1)
        km.fit(df_enc)
        cost.append(km.cost_)

import numpy as np
y = np.array([i for i in range(2,11,1)])
import matplotlib.pyplot as plt

plt.plot(y,cost)
plt.show()

Iteration number: 2
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 4310, cost: 992366.0
Iteration number: 3
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 4155, cost: 929700.0
Iteration number: 4
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 3987, cost: 902859.0
Iteration number: 5
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 23037, cost: 871979.0
Iteration number: 6
Init: initializing centroids
Init: initializing clusters
Starting iterations...


In [6]:
#choose k=3
km = KModes(n_clusters=3, n_init = 1, verbose=1)
clusters = km.fit_predict(df_enc)

print('done with kmodes')





Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 4131, cost: 1050546.0
done with kmodes


In [8]:
df_renc = df_enc.apply(le.inverse_transform)

df_ind_res = df_renc.reset_index()

ValueError: y contains previously unseen labels: [269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286
 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304
 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322
 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340
 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358
 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376
 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394
 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412
 413 414 415 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430
 431 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447 448
 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 466
 467 468 469 470 471 472 473 474 475 476 477 478 479 480 481 482 483 484
 485 486 487 488 489 490 491 492 493 494 495 496 497 498 499 500 501 502
 5

In [ ]:
clustersDf = pd.DataFrame(clusters)
clustersDf.columns = ['cluster_predicted']
df_clusters = pd.concat([clustersDf, df_ind_res], axis = 1).reset_index()
df_clusters = df_clusters.drop(['index', 'level_0'], axis = 1)

print(df_clusters.head())
for col in df_clusters.columns:
    print(col)
import matplotlib.pyplot as plt
import seaborn as sns
cols_clusters = ['source','device', 'operative_system','converted','city','hour','weekday']


for col in df_enc.columns:
    #plt.subplots(figsize = (15,5))
    sns.countplot(x=df_clusters[col],order=df_clusters[col].value_counts().index,hue=df_clusters['cluster_predicted'])
    plt.show()